In [1]:
import requests
import pandas as pd

In [ ]:
# 1. Download USGS Excel
url = "https://pubs.usgs.gov/ds/140/copper/ds140-copper-2020.xlsx"
r = requests.get(url)

if r.status_code == 200:
	with open("ds140-copper-2020.xlsx", "wb") as f:
		f.write(r.content)
	# 2. Load the sheet and inspect
	df = pd.read_excel("ds140-copper-2020.xlsx", sheet_name="Copper", engine="openpyxl")
	print(df.head())

	# 3. Extract US mine production (example: column name "Mine prod.")
	us_prod = df[["Year", "Mine prod."]].dropna()
	us_prod.columns = ["year", "us_mine_prod_kt"]
	print(us_prod.tail(10))
else:
	print(f"Failed to download file: HTTP {r.status_code}")
	df = None


Failed to download file: HTTP 404


In [24]:
import os
file_path = "../Data/ds140-copper-2020.xlsx"
# Check if the file exists and is not empty
if os.path.exists(file_path) and os.path.getsize(file_path) > 0:
	try:
		xls = pd.ExcelFile(file_path, engine="openpyxl")
		# Show sheet names to explore
		print("Available sheets:", xls.sheet_names)
		# Read main production data (assume it's in the first or second sheet)
		df_raw = xls.parse(xls.sheet_names[0], skiprows=0)
		# Preview
		print(df_raw.head())
	except Exception as e:
		print(f"Error reading Excel file: {e}")
else:
	print(f"File '{file_path}' does not exist or is empty. Please check the download step.")


Available sheets: ['Copper', 'Sheet1']
   Year  Primary production  Secondary production  New scrap  Refinery scrap  \
0  1900              291000                   NaN        NaN             NaN   
1  1901              302000                   NaN        NaN             NaN   
2  1902              317000                   NaN        NaN             NaN   
3  1903              331000                   NaN        NaN             NaN   
4  1904              404000                   NaN        NaN             NaN   

   Imports  Exports  Stocks  Consumption  Apparent consumption  \
0    28000   154000   76000       166000                166000   
1    32000    88000  128000       118000                194000   
2    51000   161000   71000       265000                264000   
3    60000   141000   80000       241000                241000   
4    65000   257000   43000       248000                249000   

   Unit value ($/t)  Unit value (98$/t)  World production  
0               357    

In [26]:

file_path = "../Data/mcs2024-coppe_salient.csv"

# Check if the file exists and is not empty
if os.path.exists(file_path) and os.path.getsize(file_path) > 0:
    try:
        df_raw = pd.read_csv(file_path)
        print("CSV loaded successfully.")
        print(df_raw.head())
    except Exception as e:
        print(f"Error reading CSV file: {e}")
else:
    print(f"File '{file_path}' does not exist or is empty. Please check the path.")


CSV loaded successfully.
  DataSource Commodity    Year  USprod_Mine_kt  USprod_Refinery-primary_kt  \
0    MCS2024    Copper  2019.0          1260.0                       985.0   
1    MCS2024    Copper  2020.0          1200.0                       872.0   
2    MCS2024    Copper  2021.0          1230.0                       922.0   
3    MCS2024    Copper  2022.0          1230.0                       912.0   
4    MCS2024    Copper  2023.0          1100.0                       850.0   

   USprod_Refinery-secondary_kt USprod_Post-consumer_kt  Imports_Ores_kt  \
0                          44.0                     166             27.0   
1                          43.0                     161              2.0   
2                          49.0                     157             11.0   
3                          40.0                    e150             12.0   
4                          40.0                     150              4.0   

   Imports_Refined_kt  Exports_Ores_kt  ...  Cons

In [29]:
# Example: select and rename columns as needed from df_raw
# (adjust column names as appropriate for your actual DataFrame)
# Example: If you want to keep only a subset of columns and rename them
# Here, we select some columns and rename for clarity
df_raw = df_raw.rename(columns={
	"USprod_Refinery-primary_kt": "Primary production",
	"USprod_Refinery-secondary_kt": "Refinery scrap",
	"USprod_Post-consumer_kt": "Secondary production",
	"Imports_Refined_kt": "Imports",
	"Exports_Ores_kt": "Exports",
	"Stocks_kt": "Stocks",
	"Consump_Reported_kt": "Consumption",
	"Consump_Apparent_kt": "Apparent consumption"
})

# Select only the columns you want to work with
df_selected = df_raw[[
	"Year",
	"us_refinery_primary_kt",
	"us_post_consumer_kt",
	"us_refinery_secondary_kt",
	"imports_refined_kt",
	"exports_ores_kt",
	"stocks_kt",
	"consump_reported_kt",
	"consump_apparent_kt"
]]

df_selected.head()

,Year,us_refinery_primary_kt,us_post_consumer_kt,us_refinery_secondary_kt,imports_refined_kt,exports_ores_kt,stocks_kt,consump_reported_kt,consump_apparent_kt
0,2019.0,985.0,166,44.0,663.0,356.0,110.0,1810.0,1820.0
1,2020.0,872.0,161,43.0,676.0,383.0,118.0,1680.0,1660.0
2,2021.0,922.0,157,49.0,919.0,344.0,117.0,1750.0,1950.0
3,2022.0,912.0,e150,40.0,732.0,353.0,83.0,1720.0,1800.0
4,2023.0,850.0,150,40.0,890.0,350.0,100.0,1700.0,1800.0


In [38]:
df_raw['Unit value ($/t)'] = df_raw['Price_US_ctslb'] * 22.0462 
# Use the just-calculated column for the adjustment
df_raw['Unit value (98$/t)'] = df_raw['Unit value ($/t)'] * (98 / 100)  # if adjusting to 98$ base
print(df_raw[['Unit value (98$/t)', 'Unit value ($/t)']])


   Unit value (98$/t)  Unit value ($/t)
0         6049.477280         6172.9360
1         6200.714212         6327.2594
2         9333.479232         9523.9584
3         8879.768436         9060.9882
4         8642.110400         8818.4800
5                 NaN               NaN


In [39]:
# CPI values
cpi_base_1998 = 163.0
cpi_by_year = {
    2020: 258.8,
    2021: 271.0,
    2022: 292.7,
    2023: 303.3,
    2024: 310.0,  # estimated
}

# Example DataFrame with 'Year' and 'UnitValue_USD_t'
df_raw['CPI'] = df_raw['Year'].map(cpi_by_year)
df_raw['Unit value (98$/t)'] = df_raw['Unit value ($/t)'] * (cpi_base_1998 / df_raw['CPI'])


In [40]:
df_raw.head

<bound method NDFrame.head of   DataSource Commodity    Year  USprod_Mine_kt  us_refinery_primary_kt  \
0    MCS2024    Copper  2019.0          1260.0                   985.0   
1    MCS2024    Copper  2020.0          1200.0                   872.0   
2    MCS2024    Copper  2021.0          1230.0                   922.0   
3    MCS2024    Copper  2022.0          1230.0                   912.0   
4    MCS2024    Copper  2023.0          1100.0                   850.0   
5        NaN       NaN     NaN             NaN                     NaN   

   us_refinery_secondary_kt us_post_consumer_kt  Imports_Ores_kt  \
0                      44.0                 166             27.0   
1                      43.0                 161              2.0   
2                      49.0                 157             11.0   
3                      40.0                e150             12.0   
4                      40.0                 150              4.0   
5                       NaN                

In [ ]:
ca_df = pd.read_csv("copper_canada_monthly.csv")  # from StatCan export
ca_annual = ca_df.groupby("year")["production_tonnes"].sum().reset_index()
ca_annual["production_kt"] = ca_annual["production_tonnes"] / 1e3

In [ ]:
combined = pd.merge(us_prod, ca_annual, on="year", how="outer").fillna(0)
combined["north_america_prod_kt"] = combined["us_mine_prod_kt"] + combined["production_kt"]
